**Библиотеки**

In [2]:
import numpy as np
import pandas as pd

**Данные**

In [3]:
data = pd.read_csv(r'data/golf_binary.csv')

In [4]:
data

,Outlook,Temperature,Humidity,Wind,Play Golf
0,Rainy,Hot,High,Weak,No
1,Rainy,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Sunny,Mild,High,Weak,Yes
4,Sunny,Cool,Normal,Weak,Yes
5,Sunny,Cool,Normal,Strong,No
6,Overcast,Cool,Normal,Strong,Yes
7,Rainy,Mild,High,Weak,No
8,Rainy,Cool,Normal,Weak,Yes
9,Sunny,Mild,Normal,Weak,Yes


In [9]:
humidity_d = {'High': 0, 'Normal': 1}
wind_d = {'Weak': 0, 'Strong': 1}
gold_d = {'No': 0, 'Yes': 1}

In [75]:
ohe_data = data[['Outlook', 'Temperature']].copy()
ohe_data = pd.get_dummies(ohe_data)
columns = ['OO', 'OR', 'OS','TC', 'TH', 'TM']
ohe_data.columns = columns
ohe_data['Humidity'] = data['Humidity'].map(humidity_d)
ohe_data['Wind'] = data['Wind'].map(wind_d)
ohe_data['Play Golf'] = data['Play Golf'].map(gold_d)

In [76]:
ohe_data

,OO,OR,OS,TC,TH,TM,Humidity,Wind,Play Golf
0,0,1,0,0,1,0,0,0,0
1,0,1,0,0,1,0,0,1,0
2,1,0,0,0,1,0,0,0,1
3,0,0,1,0,0,1,0,0,1
4,0,0,1,1,0,0,1,0,1
5,0,0,1,1,0,0,1,1,0
6,1,0,0,1,0,0,1,1,1
7,0,1,0,0,0,1,0,0,0
8,0,1,0,1,0,0,1,0,1
9,0,0,1,0,0,1,1,0,1


**Модель**

In [130]:
'Outlook', 'Temperature', 'Humidity', 'Wind', 'Play Golf'

Index(['Outlook', 'Temperature', 'Humidity', 'Wind', 'Play Golf'], dtype='object')

In [247]:
# TODO если знаменталь считается верно, то поставь вместо pcy1 = pcy0 = 1
# вероятности P(y=1), P(y=0) чтобы сократить вычисления
def bayes_predict(x, X, y):
    # x, y - NumPy arrays
    # X - NumPy matrix
    py1 = y.sum()/y.shape[0] # [P(y=1)]
    py0 = 1 - py1  # [P(y=0)]
    ty1, ty0 = X[y==1], X[y==0]  # наборы объектов при [y=1]; [y=0]
    s1, s0 = ty1.shape[0], ty0.shape[0]
    pcy1 = pcy0 = 1  # условные вероятности P(X|y=1); P(X|y=0)
    print('x =', x)  # delete it
    print('---------------------')  # delete it
    usl_ver_y1 = []  # delete it
    usl_ver_y0 = []  # delete it
    for j in range(X.shape[1]):
        print(ohe_data.columns[j])  # delete it
        pcy1 *= (ty1[:, j] == x[j]).sum() / s1  # P(X | y=1)
        pcy0 *= (ty0[:, j] == x[j]).sum() / s0  # P(X | y=0)
        a1 = (ty1[:, j] == x[j]).sum()  # delete it
        b1 = s1  # delete it
        c1 = np.round(a1/b1, 3)  # delete it
        a0 = (ty0[:, j] == x[j]).sum()  # delete it
        b0 = s0  # delete it
        c0 = a0 / b0  # delete it
        print('P(x={} | y=1): '.format(x[j]), '{}/{}={}'.format(a1,b1,c1))  # delete it
        print('P(x={} | y=0): '.format(x[j]), '{}/{}={}'.format(a0,b0,c0))  # delete it
        usl_ver_y1.append(np.round(c1, 2))  # delete it
        usl_ver_y0.append(np.round(c0, 2))  # delete it
        print('---------------------')  # delete it
    chisl = py1*pcy1  # числитель
    print('\n---|Числитель:|---')
    print('P(y=1) P(X | y=1) =')
    print('= P(y=1) P(x1={} | y=1) P(x2={} | y=1) ... P(x8={} | y=1) ='.format(x[0],x[1],x[-1]))
    py1 = np.round(y.sum()/y.shape[0], 2)
    py0 = 1 - py1
    print('= {} {} {} {} {} {} {} {} {} = {}'.format(py1,
                                                     usl_ver_y1[0],
                                                     usl_ver_y1[1],
                                                     usl_ver_y1[2],
                                                     usl_ver_y1[3],
                                                     usl_ver_y1[4],
                                                     usl_ver_y1[5],
                                                     usl_ver_y1[6],
                                                     usl_ver_y1[7],
                                                     np.round(chisl, 7)))
    print('\n---|Знаменатель:|---')
    print('P(X) = P(X | y=1) + P(X | y=0) =')
    print('= {} + P(X | y=0) ='.format(np.round(chisl, 7)))
    print('= {} + P(y=0) P(x1={} | y=0) P(x2={} | y=0) ... P(x8={} | y=0) ='.format(np.round(chisl, 7),
                                                                                    x[0],x[1],x[-1]))
    print('= {} + {} {} {} {} {} {} {} {} {} = '.format(np.round(chisl, 7),
                                                        py0,
                                                        usl_ver_y0[0],
                                                        usl_ver_y0[1],
                                                        usl_ver_y0[2],
                                                        usl_ver_y0[3],
                                                        usl_ver_y0[4],
                                                        usl_ver_y0[5],
                                                        usl_ver_y0[6],
                                                        usl_ver_y0[7]))
    print('= {} + {} = {}'.format(np.round(chisl, 7),
                                  np.round(py0*pcy0, 7),
                                  np.round(py1*pcy1 + py0*pcy0, 7)))
    znam = py1*pcy1 + py0*pcy0  # знаменатель
    print('\n---|Значение условной вероятности:|---')
    print('P(Y | X) = {} / {} = {}'.format(np.round(chisl, 7),
                                           np.round(znam, 7),
                                           np.round(chisl / znam, 3)))
    return chisl / znam

In [248]:
v = ohe_data.values
X, y = v[:, :-1], v[:, -1]

In [249]:
x = np.array([0, 1, 0, 0, 1, 0, 1, 1])

In [250]:
t = bayes_predict(x, X, y)
t

x = [0 1 0 0 1 0 1 1]
---------------------
OO
P(x=0 | y=1):  5/9=0.556
P(x=0 | y=0):  5/5=1.0
---------------------
OR
P(x=1 | y=1):  2/9=0.222
P(x=1 | y=0):  3/5=0.6
---------------------
OS
P(x=0 | y=1):  6/9=0.667
P(x=0 | y=0):  3/5=0.6
---------------------
TC
P(x=0 | y=1):  6/9=0.667
P(x=0 | y=0):  4/5=0.8
---------------------
TH
P(x=1 | y=1):  2/9=0.222
P(x=1 | y=0):  2/5=0.4
---------------------
TM
P(x=0 | y=1):  5/9=0.556
P(x=0 | y=0):  3/5=0.6
---------------------
Humidity
P(x=1 | y=1):  6/9=0.667
P(x=1 | y=0):  1/5=0.2
---------------------
Wind
P(x=1 | y=1):  3/9=0.333
P(x=1 | y=0):  3/5=0.6
---------------------

---|Числитель:|---
P(y=1) P(X | y=1) =
= P(y=1) P(x1=0 | y=1) P(x2=1 | y=1) ... P(x8=1 | y=1) =
= 0.64 0.56 0.22 0.67 0.67 0.22 0.56 0.67 0.33 = 0.0009677

---|Знаменатель:|---
P(X) = P(X | y=1) + P(X | y=0) =
= 0.0009677 + P(X | y=0) =
= 0.0009677 + P(y=0) P(x1=0 | y=0) P(x2=1 | y=0) ... P(x8=1 | y=0) =
= 0.0009677 + 0.36 1.0 0.6 0.6 0.8 0.4 0.6 0.2 0.6 = 
= 0

0.24502930388149585

**Пример**

Рассмотрим пример с некоторыми входными данными:

In [119]:
# Rainy | Cool | Humidity | Wind
x = np.array([0, 1, 0, 1, 0, 0, 1, 1])
bayes_predict(x, X, y)

0.603882954977772

In [120]:
# Sunny | Mild | No Humitidy | No Wind
x = np.array([0, 0, 1, 0, 0, 1, 0, 0])
bayes_predict(x, X, y)

0.5745798447034037

In [121]:
# Sunny | Hot | No Humitidy | Wind
x = np.array([0, 0, 1, 0, 1, 0, 0, 1])
bayes_predict(x, X, y)

0.13851608625460957